In [1]:
import os
import pathlib
from dotenv import load_dotenv

# Load some standard environment variables from a dot-env file, if it exists.
# If no such file can be found, does not fail, and so allows these environment vars to
# be populated in some other way
dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

Set session variable CATALOG to make query terms much more compact

In [2]:
import trino
from sqlalchemy.engine import create_engine

sqlstring = 'trino://{user}@{host}:{port}/'.format(
    user = os.environ['TRINO_USER'],
    host = os.environ['TRINO_HOST'],
    port = os.environ['TRINO_PORT']
)
sqlargs = {
    'auth': trino.auth.JWTAuthentication(os.environ['TRINO_PASSWD']),
    'http_scheme': 'https',
    'catalog': 'osc_datacommons_dev'
}
engine = create_engine(sqlstring, connect_args = sqlargs)
connection = engine.connect()

The EPA's GHG Reporting Program (GHGRP) seems to be a gold standard in terms of creating a bottoms-up list that's good enough to play a major role in tops-down estimates.

In [3]:
qres = engine.execute(f"show tables in epa_ghgrp")
display(qres.fetchall())

[('co2_injection',),
 ('direct_emitters',),
 ('gathering_boosting',),
 ('geologic_sequestration_of_co2',),
 ('ldc_direct_emissions',),
 ('onshore_oil_gas_prod',),
 ('parent_company',),
 ('sf6_from_elec_equip',),
 ('suppliers',),
 ('transmission_pipelines',)]

Direct Emitters are the lion's share of CO2 emissions.  There are other activities and industries with non-trivial emissions, but nothing compares to this.

In [4]:
qres = engine.execute(f"describe epa_ghgrp.direct_emitters")
display(qres.fetchall())

[('facility_id', 'bigint', '', ''),
 ('frs_id', 'varchar', '', ''),
 ('facility_name', 'varchar', '', ''),
 ('city', 'varchar', '', ''),
 ('state', 'varchar', '', ''),
 ('zip_code', 'varchar', '', ''),
 ('address', 'varchar', '', ''),
 ('county', 'varchar', '', ''),
 ('latitude', 'double', '', ''),
 ('longitude', 'double', '', ''),
 ('primary_naics_code', 'varchar', '', ''),
 ('latest_reported_industry_type_subparts', 'varchar', '', ''),
 ('latest_reported_industry_type_sectors', 'varchar', '', ''),
 ('total_reported_direct_emissions', 'double', '', ''),
 ('year', 'timestamp(3)', '', '')]

In [5]:
qres = engine.execute(f"select year, sum(total_reported_direct_emissions) from epa_ghgrp.direct_emitters group by year order by year desc")
display(qres.fetchall())

[('2020-01-01 00:00:00.000', 2400335017.1144834),
 ('2019-01-01 00:00:00.000', 2626532098.5932565),
 ('2018-01-01 00:00:00.000', 2779470637.179425),
 ('2017-01-01 00:00:00.000', 2735840754.8973103),
 ('2016-01-01 00:00:00.000', 2805104747.5443416),
 ('2015-01-01 00:00:00.000', 2939443583.3242407),
 ('2014-01-01 00:00:00.000', 3084068793.216346),
 ('2013-01-01 00:00:00.000', 3073213896.2733335),
 ('2012-01-01 00:00:00.000', 3058075792.5988026),
 ('2011-01-01 00:00:00.000', 3207582994.0028186)]

Here's a look at how they stack up (from a Database perspective--we should also look at this in Super Set).

In [6]:
qres = engine.execute(f"""select count (*), latest_reported_industry_type_sectors, sum(total_reported_direct_emissions)
from epa_ghgrp.direct_emitters
where year>=DATE('2019-01-01') and year<DATE('2020-01-01')
group by latest_reported_industry_type_sectors
order by sum(total_reported_direct_emissions) desc
""")
display(qres.fetchall())

[(1338, 'Power Plants', 1593302651.1495945),
 (371, 'Minerals', 112044493.16399996),
 (328, 'Chemicals', 106404264.83600003),
 (1066, 'Other', 101353022.50774214),
 (1187, 'Petroleum and Natural Gas Systems', 101032012.85843231),
 (1204, 'Waste', 100624446.79977302),
 (281, 'Metals', 69494113.83299993),
 (69, 'Petroleum Product Suppliers,Refineries', 65079994.47800001),
 (41, 'Chemicals,Petroleum Product Suppliers,Refineries', 64019038.494),
 (29, 'Chemicals,Suppliers of CO2', 27435824.12000001),
 (84, 'Pulp and Paper,Waste', 24953431.80600001),
 (7, 'Power Plants,Waste', 24875284.23),
 (11, 'Chemicals,Waste', 17522490.718),
 (10, 'Other,Power Plants', 17315702.882),
 (112, 'Other,Waste', 17100924.523999985),
 (3, 'Power Plants,Suppliers of CO2', 14904814.286),
 (17, 'Metals,Waste', 14615880.466000002),
 (4, 'Chemicals,Refineries,Suppliers of CO2', 13389032.404000001),
 (132, 'Pulp and Paper', 13028976.956000002),
 (97, 'Natural Gas and Natural Gas Liquids Suppliers,Petroleum and Natur

This looks at the `Metals` industry.  We see that the top emitters have multiple facility locations.

In [7]:
qres = engine.execute(f"""select count (*), parent_company_name, sum(total_reported_direct_emissions)
from epa_ghgrp.direct_emitters, epa_ghgrp.parent_company
where year>=DATE('2019-01-01') and year<DATE('2020-01-01') and year=reporting_year
      and latest_reported_industry_type_sectors='Minerals'
      and epa_ghgrp.direct_emitters.facility_id=epa_ghgrp.parent_company.ghgrp_facility_id
group by parent_company_name
order by sum(total_reported_direct_emissions) desc
limit 20
""")
display(qres.fetchall())

[(12, 'HOLCIM PARTICIPATIONS (US) INC', 10775100.462000001),
 (9, 'CEMEX INC', 7837412.011999999),
 (12, 'LEHIGH HANSON INC', 7170874.248),
 (8, 'RC LONESTAR INC', 6670842.868),
 (11, 'LHOIST NORTH AMERICA INC', 6095893.518),
 (11, 'CARMEUSE LIME INC', 4957669.602),
 (9, 'GRAYMONT INC', 4145991.5659999996),
 (8, 'CRH AMERICAS INC', 4038046.516),
 (4, 'MARTIN MARIETTA MATERIALS INC', 3939577.874000001),
 (4, 'ARGOS USA LLC', 3571057.906),
 (11, 'EAGLE MATERIALS INC', 3460129.46),
 (3, 'MISSISSIPPI LIME CO', 3386972.9599999995),
 (3, 'TAIHEIYO CEMENT USA INC', 3268649.0960000004),
 (5, 'GCC OF AMERICA INC', 2311521.19),
 (2, 'TITAN AMERICA LLC', 2230059.0100000002),
 (1, 'GENESIS ENERGY LP', 1824295.1),
 (1, 'LAFARGEHOLCIM NORTH AMERICA INC', 1792093.716),
 (2, 'NATIONAL CEMENT', 1633243.352),
 (2, 'SUMMIT MATERIALS INC', 1555637.794),
 (1, 'TATA CHEMICALS (SODA ASH) PARTNERS NORTH AMERICA', 1536763.272)]

A look at some other table shapes...

In [8]:
qres = engine.execute(f"describe epa_ghgrp.onshore_oil_gas_prod")
display(qres.fetchall())

[('facility_id', 'bigint', '', ''),
 ('frs_id', 'varchar', '', ''),
 ('facility_name', 'varchar', '', ''),
 ('basin', 'varchar', '', ''),
 ('reported_city', 'varchar', '', ''),
 ('reported_state', 'varchar', '', ''),
 ('reported_zip_code', 'varchar', '', ''),
 ('reported_address', 'varchar', '', ''),
 ('reported_county', 'varchar', '', ''),
 ('reported_latitude', 'double', '', ''),
 ('reported_longitude', 'double', '', ''),
 ('primary_naics_code', 'varchar', '', ''),
 ('latest_reported_industry_type_subparts', 'varchar', '', ''),
 ('total_reported_emissions_from_onshore_oil_gas_production', 'double', '', ''),
 ('year', 'timestamp(3)', '', '')]

In [9]:
qres = engine.execute(f"describe epa_ghgrp.suppliers")
display(qres.fetchall())

[('facility_id', 'bigint', '', ''),
 ('frs_id', 'varchar', '', ''),
 ('facility_name', 'varchar', '', ''),
 ('city', 'varchar', '', ''),
 ('state', 'varchar', '', ''),
 ('zip_code', 'varchar', '', ''),
 ('address', 'varchar', '', ''),
 ('county', 'varchar', '', ''),
 ('latitude', 'double', '', ''),
 ('longitude', 'double', '', ''),
 ('primary_naics_code', 'varchar', '', ''),
 ('latest_reported_industry_type_subparts', 'varchar', '', ''),
 ('coal_based_liquid_fuel_production_ghg', 'double', '', ''),
 ('petroleum_products_produced_ghg', 'double', '', ''),
 ('petroleum_products_imported_ghg', 'double', '', ''),
 ('petroleum_products_exported_ghg', 'double', '', ''),
 ('natural_gas_supply_ghg', 'double', '', ''),
 ('natural_gas_liquids_supply_ghg', 'double', '', ''),
 ('co2_supply_ghg', 'double', '', ''),
 ('year', 'timestamp(3)', '', '')]

A quick summary of how many rows of data we have in `epa_ghgrp`.

68k rows in `direct_emitters`: lots of facilities  
103k rows in `parent_company`: lots of facility/owner relationships

In [10]:
qres = engine.execute('show tables in osc_datacommons_dev.epa_ghgrp')
l = qres.fetchall()
totalrows = 0
for e in l:
    s = f'select count (*) from osc_datacommons_dev.epa_ghgrp.{e[0]}'
    print(s)
    qres = engine.execute(s)
    rowcount = qres.fetchall()[0][0]
    totalrows += rowcount
    print(rowcount)
    
print(f'total rows = {totalrows}')

select count (*) from osc_datacommons_dev.epa_ghgrp.co2_injection
954
select count (*) from osc_datacommons_dev.epa_ghgrp.direct_emitters
68472
select count (*) from osc_datacommons_dev.epa_ghgrp.gathering_boosting
1703
select count (*) from osc_datacommons_dev.epa_ghgrp.geologic_sequestration_of_co2
20
select count (*) from osc_datacommons_dev.epa_ghgrp.ldc_direct_emissions
1730
select count (*) from osc_datacommons_dev.epa_ghgrp.onshore_oil_gas_prod
5068
select count (*) from osc_datacommons_dev.epa_ghgrp.parent_company
103043
select count (*) from osc_datacommons_dev.epa_ghgrp.sf6_from_elec_equip
1012
select count (*) from osc_datacommons_dev.epa_ghgrp.suppliers
8539
select count (*) from osc_datacommons_dev.epa_ghgrp.transmission_pipelines
780
total rows = 191321


We have very specifically shaped tables, which is needed to prepare this presetnation in Pandas.

The key metric is total_emissions (in metric tons of CO2e), but the name of the metric depends on the source/process.  Nevertheless, we know that `year` is our last metric and that the CO2e metric is 2nd-to-last (hence the `-2` index).

We also know that when building our final summary table, the sums feeding into it are all only one row per year.  We use `iat[0,1` to access the 0th row and the 1st column (which will be named specifically to the source/process).  By using `iat`, we get a scalar value we can sum, instead of a Series object we'd have to `squeeze`.

In [11]:
import pandas as pd

emission_tables = ['direct_emitters', 'onshore_oil_gas_prod', 'gathering_boosting', 'transmission_pipelines', 'ldc_direct_emissions', 'sf6_from_elec_equip']
q_dict = {}

# A function that excludes terms using SQL to say "and X!=Y"
def excl_text(excl):
    return ' and '.join([f"latest_reported_industry_type_sectors!='{e}'" for e in excl])

# A function that includes text that matches; SQL that says "or X like '%Y%'"
def incl_text(excl):
    return ' or '.join([f"latest_reported_industry_type_sectors like '%{e}%'" for e in excl])

t = 'direct_emitters'
qres = engine.execute(f"describe epa_ghgrp.{t}")
t_cols = qres.fetchall()
total_emission_cname = t_cols[-2][0]

incl = [ 'Power', 'Petroleum']
qres = engine.execute(f"""
select year, sum({total_emission_cname}) from epa_ghgrp.{t}
where {incl_text(incl)}
group by year
""")
q_dict[t + f" (incl {','.join(incl)})"] = pd.DataFrame(qres.fetchall(), columns=['year', total_emission_cname + f" (matching {','.join(incl)})"])

excl = [ 'Minerals', 'Other', 'Waste', 'Chemicals', 'Pulp and Paper,Waste', 'Metals,Waste', 'Pulp and Paper']
qres = engine.execute(f"""
select year, sum({total_emission_cname}) from epa_ghgrp.{t}
where {excl_text(excl)}
group by year
""")
q_dict[t + f" (excl {','.join(excl)})"] = pd.DataFrame(qres.fetchall(), columns=['year', total_emission_cname + f" (excl {','.join(excl)})"])

for t in emission_tables:
    qres = engine.execute(f"describe epa_ghgrp.{t}")
    tr = qres.fetchall()
    total_emission_cname = tr[-2][0]
    qres = engine.execute(f"select year, sum({total_emission_cname}) from epa_ghgrp.{t} group by year")
    q_dict[t] = pd.DataFrame(qres.fetchall(), columns=['year', total_emission_cname])

grand_total = {}

for year in q_dict['direct_emitters'].year:
    grand_total[year] = sum([q_dict[t][q_dict[t].year==year].iat[0,1] for t in emission_tables if year in q_dict[t].year.values])

df = pd.DataFrame.from_dict(grand_total, orient='index', columns=['total_co2e']).reset_index()
df.rename(columns={'index':'year'}, inplace=True)
q_dict['grand_total'] = df

This gem comes from https://stackoverflow.com/questions/44327999/python-pandas-merge-multiple-dataframes

In [12]:
from functools import reduce

df_merged = reduce(lambda left,right: pd.merge(left,right,on=['year'], how='outer'), q_dict.values()).fillna(0)

A summary table showing (1) direct emitters that match "Power" or "Petroleum", (2) direct emitters that are not the top other industrie, and then (3..N): the totals from the GHGRP.

In [13]:
df_merged.rename(columns={v:v.replace('_', ' ') for v in df_merged.columns.values}).sort_values(by='year', ascending=False)

,year,"total reported direct emissions (matching Power,Petroleum)","total reported direct emissions (excl Minerals,Other,Waste,Chemicals,Pulp and Paper,Waste,Metals,Waste,Pulp and Paper)",total reported direct emissions,total reported emissions from onshore oil gas production,total reported emissions from gathering boosting,total reported direct emissions from transmission pipelines,total reported direct emissions from local dist companies,total reported direct emissions from electrical equipment use,total co2e
4,2020-01-01 00:00:00.000,1.768353e+09,1.947463e+09,2.400335e+09,9.348811e+07,9.002867e+07,3.497590e+06,1.264110e+07,2.004836e+06,2.601995e+09
3,2019-01-01 00:00:00.000,1.958369e+09,2.153508e+09,2.626532e+09,1.201743e+08,9.276566e+07,2.859475e+06,1.284702e+07,2.510832e+06,2.857689e+09
2,2018-01-01 00:00:00.000,2.099221e+09,2.304343e+09,2.779471e+09,1.119588e+08,8.332560e+07,3.050315e+06,1.323626e+07,2.270228e+06,2.993312e+09
6,2017-01-01 00:00:00.000,2.070082e+09,2.270354e+09,2.735841e+09,9.624146e+07,7.783058e+07,2.699047e+06,1.367043e+07,2.555766e+06,2.928838e+09
0,2016-01-01 00:00:00.000,2.144348e+09,2.337524e+09,2.805105e+09,8.689825e+07,8.259701e+07,3.183982e+06,1.400229e+07,2.930497e+06,2.994717e+09
1,2015-01-01 00:00:00.000,2.261725e+09,2.452045e+09,2.939444e+09,1.017485e+08,0.000000e+00,0.000000e+00,1.455831e+07,2.472281e+06,3.058223e+09
7,2014-01-01 00:00:00.000,2.392070e+09,2.592859e+09,3.084069e+09,1.019517e+08,0.000000e+00,0.000000e+00,1.477185e+07,3.220287e+06,3.204013e+09
9,2013-01-01 00:00:00.000,2.392082e+09,2.593335e+09,3.073214e+09,9.795946e+07,0.000000e+00,0.000000e+00,1.516147e+07,3.258298e+06,3.189593e+09
5,2012-01-01 00:00:00.000,2.376107e+09,2.579361e+09,3.058076e+09,9.253966e+07,0.000000e+00,0.000000e+00,1.541235e+07,3.236291e+06,3.169264e+09
8,2011-01-01 00:00:00.000,2.509918e+09,2.728839e+09,3.207583e+09,9.119057e+07,0.000000e+00,0.000000e+00,1.566794e+07,3.920547e+06,3.318362e+09


Cross-check with ESSD tops-down dataset

In [14]:
qres = engine.execute('describe essd.ghg_data')
qres.fetchall()

[('iso', 'varchar', '', ''),
 ('country', 'varchar', '', ''),
 ('region_ar6_6', 'varchar', '', ''),
 ('region_ar6_10', 'varchar', '', ''),
 ('region_ar6_22', 'varchar', '', ''),
 ('region_ar6_dev', 'varchar', '', ''),
 ('sector_title', 'varchar', '', ''),
 ('subsector_title', 'varchar', '', ''),
 ('gas', 'varchar', '', ''),
 ('gwp100_ar5', 'integer', '', ''),
 ('value', 'double', '', ''),
 ('year', 'timestamp(3)', '', '')]

A quick look at *just* CO2.  We'll look at CO2e in the next set of cells.

In [15]:
qres = engine.execute("select year, sum(value), gas from essd.ghg_data where sector_title='Energy systems' and gas='CO2' and year>DATE('2010-01-01') and ISO='USA' group by year, gas order by year desc")
qres.fetchall()

[('2020-01-01 00:00:00.000', 1751743410.33274, 'CO2'),
 ('2019-01-01 00:00:00.000', 1987569148.7149098, 'CO2'),
 ('2018-01-01 00:00:00.000', 2130195623.42906, 'CO2'),
 ('2017-01-01 00:00:00.000', 2106892474.32796, 'CO2'),
 ('2016-01-01 00:00:00.000', 2180627611.48211, 'CO2'),
 ('2015-01-01 00:00:00.000', 2275835961.04433, 'CO2'),
 ('2014-01-01 00:00:00.000', 2411676648.3933206, 'CO2'),
 ('2013-01-01 00:00:00.000', 2410360932.0616, 'CO2'),
 ('2012-01-01 00:00:00.000', 2390450900.39202, 'CO2'),
 ('2011-01-01 00:00:00.000', 2514780563.17824, 'CO2')]

In [16]:
qres = engine.execute('describe essd.gwp100_data')
qres.fetchall()

[('iso', 'varchar', '', ''),
 ('country', 'varchar', '', ''),
 ('region_ar6_6', 'varchar', '', ''),
 ('region_ar6_10', 'varchar', '', ''),
 ('region_ar6_22', 'varchar', '', ''),
 ('region_ar6_dev', 'varchar', '', ''),
 ('sector_title', 'varchar', '', ''),
 ('subsector_title', 'varchar', '', ''),
 ('co2', 'double', '', ''),
 ('ch4', 'double', '', ''),
 ('n2o', 'double', '', ''),
 ('fgas', 'double', '', ''),
 ('ghg', 'double', '', ''),
 ('year', 'timestamp(3)', '', '')]

A look at CO2e (presuming that's what GHG gives us from the GWP100 table).

In [17]:
qres = engine.execute("select year, sum(GHG) from essd.gwp100_data where sector_title='Energy systems' and year>DATE('2010-01-01') and ISO='USA' group by year order by year desc")
qres.fetchall()

[('2020-01-01 00:00:00.000', 1751743410.33274),
 ('2019-01-01 00:00:00.000', 2349291999.2192674),
 ('2018-01-01 00:00:00.000', 2477543798.400163),
 ('2017-01-01 00:00:00.000', 2451547006.5038967),
 ('2016-01-01 00:00:00.000', 2514167924.00994),
 ('2015-01-01 00:00:00.000', 2634213163.051593),
 ('2014-01-01 00:00:00.000', 2784719011.4105005),
 ('2013-01-01 00:00:00.000', 2779046573.6036997),
 ('2012-01-01 00:00:00.000', 2759253824.43615),
 ('2011-01-01 00:00:00.000', 2890949806.5313897)]

In [18]:
qres = engine.execute("describe us_census.all_sector_survey_2017")
display(qres.fetchall())
qres = engine.execute("select * from us_census.all_sector_survey_2017 where naics2012='221112'")
display(qres.fetchall())


[('geo_id', 'varchar', '', ''),
 ('name', 'varchar', '', ''),
 ('geo_id_f', 'varchar', '', ''),
 ('naics2012', 'varchar', '', ''),
 ('naics2012_f', 'varchar', '', ''),
 ('naics2012_label', 'varchar', '', ''),
 ('year', 'varchar', '', ''),
 ('estab', 'varchar', '', ''),
 ('rcptot', 'varchar', '', ''),
 ('payann', 'varchar', '', ''),
 ('emp', 'varchar', '', '')]

[('0100000US', 'United States', None, '221112', None, 'Fossil fuel electric power generation', '2012', '1416', '81473633', '7997908', '82071'),
 ('0100000US', 'United States', None, '221112', None, 'Fossil fuel electric power generation', '2017', '1711', '75455040', '8192622', '76058')]

Exercise the connection to NAICS and sector information provided by US Department of Commerce (US_CENSUS)

In [19]:
# qres = engine.execute("select count (*), epa_ghgrp.direct_emitters.year, primary_naics_code from epa_ghgrp.direct_emitters where primary_naics_code='221112' group by epa_ghgrp.direct_emitters.year, primary_naics_code order by count (*) desc limit 20")
# display(qres.fetchall())

qres = engine.execute(f"""
select count (*), epa_ghgrp.direct_emitters.year, primary_naics_code, naics2012_label
from epa_ghgrp.direct_emitters, us_census.all_sector_survey_2017
where primary_naics_code=naics2012 and us_census.all_sector_survey_2017.year='2017'
group by epa_ghgrp.direct_emitters.year, primary_naics_code, naics2012_label order by count (*) desc limit 20
""")
display(qres.fetchall())

[(1483, '2012-01-01 00:00:00.000', '221112', 'Fossil fuel electric power generation'),
 (1467, '2011-01-01 00:00:00.000', '221112', 'Fossil fuel electric power generation'),
 (1454, '2013-01-01 00:00:00.000', '221112', 'Fossil fuel electric power generation'),
 (1425, '2014-01-01 00:00:00.000', '221112', 'Fossil fuel electric power generation'),
 (1368, '2015-01-01 00:00:00.000', '221112', 'Fossil fuel electric power generation'),
 (1304, '2016-01-01 00:00:00.000', '221112', 'Fossil fuel electric power generation'),
 (1294, '2018-01-01 00:00:00.000', '221112', 'Fossil fuel electric power generation'),
 (1281, '2017-01-01 00:00:00.000', '221112', 'Fossil fuel electric power generation'),
 (1274, '2019-01-01 00:00:00.000', '221112', 'Fossil fuel electric power generation'),
 (1247, '2012-01-01 00:00:00.000', '562212', 'Solid waste landfill'),
 (1244, '2020-01-01 00:00:00.000', '221112', 'Fossil fuel electric power generation'),
 (1235, '2011-01-01 00:00:00.000', '562212', 'Solid waste la

See how many `PARENT_COMPANY` records have LEIs we know.  Note that there are about 5x facilities for each known LEI.  There are fewer average facilities for parent copmanies with unknown LEIs.

In [20]:
qres = engine.execute(f"select count (*), reporting_year from (select lei, reporting_year from epa_ghgrp.parent_company where LEI is not null group by lei, reporting_year) group by reporting_year order by reporting_year desc")
qres.fetchall()

[(1542, '2020-01-01 00:00:00.000'),
 (1559, '2019-01-01 00:00:00.000'),
 (1626, '2018-01-01 00:00:00.000'),
 (1562, '2017-01-01 00:00:00.000'),
 (1560, '2016-01-01 00:00:00.000'),
 (1541, '2015-01-01 00:00:00.000'),
 (1712, '2014-01-01 00:00:00.000'),
 (1695, '2013-01-01 00:00:00.000'),
 (1657, '2012-01-01 00:00:00.000'),
 (1578, '2011-01-01 00:00:00.000'),
 (1309, '2010-01-01 00:00:00.000')]

In [21]:
qres = engine.execute(f"describe sec_dera.sub")
qres.fetchall()

[('adsh', 'varchar', '', ''),
 ('cik', 'integer', '', ''),
 ('name', 'varchar', '', ''),
 ('lei', 'varchar', '', ''),
 ('sic', 'integer', '', ''),
 ('countryba', 'varchar', '', ''),
 ('stprba', 'varchar', '', ''),
 ('cityba', 'varchar', '', ''),
 ('zipba', 'varchar', '', ''),
 ('bas1', 'varchar', '', ''),
 ('bas2', 'varchar', '', ''),
 ('baph', 'varchar', '', ''),
 ('countryma', 'varchar', '', ''),
 ('stprma', 'varchar', '', ''),
 ('cityma', 'varchar', '', ''),
 ('zipma', 'varchar', '', ''),
 ('mas1', 'varchar', '', ''),
 ('mas2', 'varchar', '', ''),
 ('countryinc', 'varchar', '', ''),
 ('stprinc', 'varchar', '', ''),
 ('ein', 'bigint', '', ''),
 ('former', 'varchar', '', ''),
 ('changed', 'varchar', '', ''),
 ('afs', 'varchar', '', ''),
 ('wksi', 'boolean', '', ''),
 ('fye', 'varchar', '', ''),
 ('form', 'varchar', '', ''),
 ('period', 'timestamp(3)', '', ''),
 ('fy', 'timestamp(3)', '', ''),
 ('fp', 'varchar', '', ''),
 ('filed', 'timestamp(3)', '', ''),
 ('accepted', 'timestamp(3)',

In [22]:
qres = engine.execute(f"describe sec_dera.num")
qres.fetchall()

[('adsh', 'varchar', '', ''),
 ('tag', 'varchar', '', ''),
 ('version', 'varchar', '', ''),
 ('coreg', 'varchar', '', ''),
 ('ddate', 'timestamp(3)', '', ''),
 ('qtrs', 'integer', '', ''),
 ('uom', 'varchar', '', ''),
 ('value', 'double', '', ''),
 ('footnote', 'varchar', '', ''),
 ('uuid', 'varchar', '', ''),
 ('quarter', 'varchar', '', '')]

In [23]:
stop!

SyntaxError: invalid syntax (<ipython-input-23-8c726f94684c>, line 1)

Things start getting ugly here.  The `FACILITY_ID` is a primary key in tables like `DIRECT_EMITTERS`.  The Plum Point Energy Station (#1000539) has 5 owners, meaning there are 5 entries in the `parent_copmany` table with #1000539 as the `GHGRP_FACILITY_ID`.  If not careful, that plant's 3.4M Mt CO2e could be reported as 5x as much if each owner picks up the full attribution of those emissions.  Similarly, the DERA data is quarterly, which means that an interesting datapoints can occur 4 times per year.  We must take care when connecting annually-reported data to quarterly-reported data.  So...BEWARE!

Everything from the point onward are sketchy (literally) and likely wrong.

In [ ]:
qres = engine.execute(f"""select count (*), year
from epa_ghgrp.parent_company, epa_ghgrp.direct_emitters, sec_dera.sub
where epa_ghgrp.parent_company.lei is not null
and epa_ghgrp.parent_company.ghgrp_facility_id=epa_ghgrp.direct_emitters.facility_id
and year=reporting_year and reporting_year >= DATE('2020-01-01') and filed > DATE('2020-01-01') and filed < DATE('2021-01-01')
and epa_ghgrp.parent_company.lei=sec_dera.sub.lei
group by year
order by year desc
""")
qres.fetchall()

In [ ]:
qres = engine.execute(f"""select count (*), sec_dera.sub.period
from sec_dera.sub
where sec_dera.sub.lei is not null and sec_dera.sub.period>=DATE('2020-01-01') and sec_dera.sub.period<DATE('2021-01-01')
group sec_dera.sub.period
order sec_dera.sub.period""")
qres.fetchall()

In [ ]:
qres = engine.execute(f"""select count (*), reporting_year
from epa_ghgrp.parent_company, epa_ghgrp.direct_emitters
where epa_ghgrp.parent_company.lei is not null and reporting_year >= DATE('2020-01-01') and year=reporting_year
and epa_ghgrp.parent_company.ghgrp_facility_id=epa_ghgrp.direct_emitters.facility_id
group by reporting_year
""")
qres.fetchall()

In [ ]:
qres = engine.execute(f"""
select DISTINCT(sec_dera.sub.lei), ghgrp_facility_id from osc_datacommons_dev.epa_ghgrp.parent_company,sec_dera.sub where epa_ghgrp.parent_company.lei=sec_dera.sub.lei
-- and sec_dera.sub.lei is null
""")
l = cur.fetchall()
print(len(l))

In [ ]:
qres = engine.execute(f"""select * from sec_dera.ticker limit 10""")
qres.fetchall()

In [ ]:
qres = engine.execute(f"""
with leis as (select DISTINCT(sec_dera.sub.lei), if(tname IS NULL, '<private>', tname) as ticker
              from epa_ghgrp.parent_company,sec_dera.sub,sec_dera.ticker
              where epa_ghgrp.parent_company.lei=sec_dera.sub.lei and form='10-Q'
              and sec_dera.sub.cik=sec_dera.ticker.cik)
select count (*), ticker, leis.lei, parent_company_name, reporting_year
from epa_ghgrp.parent_company, epa_ghgrp.direct_emitters, leis
where reporting_year >= DATE('2020-01-01') and year=reporting_year
and epa_ghgrp.parent_company.lei=leis.lei
and epa_ghgrp.parent_company.ghgrp_facility_id=epa_ghgrp.direct_emitters.facility_id
group by leis.ticker, leis.lei, parent_company_name, reporting_year
order by count(*) desc
-- limit 10
""")
l = qres.fetchall()
print(len(l))

In [ ]:
l

In [ ]:
sum([e[0] for e in l])

In [ ]:
qres = engine.execute(f"""
with leis as (select DISTINCT(sec_dera.sub.lei), ghgrp_facility_id, adsh
              from epa_ghgrp.parent_company,sec_dera.sub
              where epa_ghgrp.parent_company.lei=sec_dera.sub.lei and form='10-Q')
select leis.lei as lei, parent_company_name, sum(total_reported_direct_emissions) as tot_co2e, reporting_year, uom, sum(value) as tot_revenue, sum(total_reported_direct_emissions)/sum(value) as intensity
from leis, epa_ghgrp.parent_company, epa_ghgrp.direct_emitters, sec_dera.num
where osc_datacommons_dev.epa_ghgrp.parent_company.ghgrp_facility_id=leis.ghgrp_facility_id
and reporting_year >= DATE('2020-01-01') and year=reporting_year
and epa_ghgrp.parent_company.ghgrp_facility_id=epa_ghgrp.direct_emitters.facility_id
and leis.adsh=sec_dera.num.adsh
and sec_dera.num.tag = 'Revenues'
and sec_dera.num.ddate < DATE('2021-01-01')
and sec_dera.num.ddate >= DATE('2020-01-01')
and sec_dera.num.qtrs=1
and total_reported_direct_emissions>0
group by leis.lei, parent_company_name, reporting_year, uom
order by intensity desc
-- limit 100
""")
list = qres.fetchall()
print(len(list))
display(list)